In [294]:
# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName('MyApp').getOrCreate()

In [295]:
movies = spark.read.csv('../datasets/imdb/train_movies_extra_data_new.csv', header=True, inferSchema=True)

In [296]:
from pyspark.sql.functions import col, udf, when, split
from pyspark.sql.types import IntegerType, BooleanType, FloatType

import re

def manage_awards(award):
    if re.search('win', award) or re.search('wins', award):
        return True
    else:
        return False

my_udf = udf(manage_awards, BooleanType())

def preprocessing_method(df):

    """
    This method will convert to integer all the years and will put to categorical weather
    a movie won any awards or not.
    :param df: dataframe
    :return: return a preprocessed dataframe
    """

    """
    Here casting to integer all the numerical values
    """
    df = df.withColumn('startYear', col('startYear').cast('int'))
    df = df.withColumn('endYear', col('endYear').cast('int'))
    df = df.withColumn('numVotes', col('numVotes').cast('int'))
    df = df.withColumn('boxoffice', col('boxoffice').cast('int'))
    df = df.withColumn('imdb_votes', col('imdb_votes').cast('int'))

    """
    Awards transformed into True or False
    """
    df = df.withColumn('awards', my_udf(df['awards']))

    """
    Substitute numVotes value when null with the value of imdb votes
    """

    df = df.withColumn('numVotes', when(col('numVotes').isNull(), col('imdb_votes')).
                       otherwise(col('numVotes')))

    """
    Transformations from string to lists
    """
    df = df.withColumn('genre', split(col('genre'), ','))
    df = df.withColumn('actors', split(col('actors'), ','))
    df = df.withColumn('language', split(col('language'), ','))

    return df

In [298]:
movies = preprocessing_method(movies)